# CIA World Fact Dataset

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type="table";

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [6]:
%%sql
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


It looks like there is some erronous values: minimum poppulation = 0 and max population 7 billion, lets investigate

In [9]:
%%sql
SELECT name, population
FROM facts
WHERE population = (
    SELECT MIN(population)
    FROM facts);

Done.


name,population
Antarctica,0


In [10]:
%%sql
SELECT name, population
FROM facts
WHERE population = (
    SELECT MAX(population)
    FROM facts);

Done.


name,population
World,7256490011


calculating again while excluding those values

In [11]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts));

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


In [12]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts));

Done.


AVG(population),AVG(area)
32377011.0125,582949.8523206752


Finding countries that are above the population average and below the area average

In [14]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts)))
AND area < (SELECT AVG(area) FROM facts WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts)))
AND population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts)); 

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
Kenya,45925301,580367
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685


## Which country has most people? Which has highest growth rate?

In [16]:
%%sql
SELECT name,  population
FROM facts
WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts))
ORDER BY population DESC
LIMIT 1;

Done.


name,population
China,1367485388


In [17]:
%%sql
SELECT name,  population
FROM facts
WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts))
ORDER BY population DESC
LIMIT 1;

Done.


name,population
China,1367485388


In [18]:
%%sql 
SELECT name, population_growth
FROM facts
WHERE population_growth = (
    SELECT MAX(population_growth) FROM facts);

Done.


name,population_growth
South Sudan,4.02


## Highest ratio of water to land, which have more water than land?

In [26]:
%%sql
SELECT name, ROUND(CAST(area_water AS FLOAT)/ CAST(area_land AS FLOAT), 3) AS ratio
FROM facts
WHERE population NOT IN (
    (SELECT MAX(population) FROM facts), (SELECT MIN(population) FROM facts))
ORDER BY ratio DESC
LIMIT 5;

Done.


name,ratio
Virgin Islands,4.52
Puerto Rico,0.555
"Bahamas, The",0.387
Guinea-Bissau,0.285
Malawi,0.259


In [28]:
%%sql
SELECT name, area_water, area_land
FROM facts
WHERE name <> "World"
AND area_water > area_land;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


## Which countries will add the most people next year?

In [ ]:
%%sql
SELECT name,
       